In [16]:
import numpy as np
import pandas as pd
import csv
import json
import matplotlib.pyplot as plt
from datetime import datetime
from math import sqrt
import numpy as np

%matplotlib inline

with open('/Users/jieyizhang/Downloads/yelp_dataset/yelp_academic_dataset_review.json') as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

review_df = pd.DataFrame(data)

with open('/Users/jieyizhang/Downloads/yelp_dataset/yelp_academic_dataset_business.json') as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

business_df = pd.DataFrame(data)

In [2]:
review_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
2,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
3,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
4,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,0,msQe1u7Z_XuqjGoqhB0J5g


In [3]:
reviews = review_df[['user_id','business_id','stars']]
reviews.shape

(5996996, 3)

In [ ]:
dataset = {}

def func(x):
    uid = x['user_id']
    bid = x['business_id']
    stars = x['stars']
    
    if uid not in dataset.keys():
        dataset[uid]= {} 
    dataset[uid][bid] = stars
    
reviews.apply(lambda x: func(x), axis=1)

def similarity_score(person1,person2):
# Returns ratio Euclidean distance score of person1 and person2 
    both_viewed = {}
    # To get both rated items by person1 and person2
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_viewed[item] = 1
        # Conditions to check they both have an common rating items	
        if len(both_viewed) == 0:
            return 0

        # Finding Euclidean distance 
        sum_of_eclidean_distance = []

        for item in dataset[person1]:
            if item in dataset[person2]:
                sum_of_eclidean_distance.append(pow(dataset[person1][item] - dataset[person2][item],2))
        sum_of_eclidean_distance = sum(sum_of_eclidean_distance)

        return 1/(1+sqrt(sum_of_eclidean_distance))
    
def pearson_correlation(person1,person2):

    # To get both rated items
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)		

    # Checking for number of ratings in common
    if number_of_ratings == 0:
        return 0

    # Add up all the preferences of each user
    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    # Sum up the squares of preferences of each user
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

    # Sum up the product value of both preferences for each item
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    # Calculate the pearson score
    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))
    if denominator_value == 0:
        return 0
    else:
        r = numerator_value/denominator_value
        return r
    
def most_similar_users(person,number_of_users):
    # returns the number_of_users (similar persons) for a given specific person.
    scores = [(pearson_correlation(person,other_person),other_person) for other_person in dataset if  other_person != person ]

    # Sort the similar persons so that highest scores person will appear at the first
    scores.sort()
    scores.reverse()
    return scores[0:number_of_users]

def user_recommendations(person, maxnum):

    # Gets recommendations for a person by using a weighted average of every other user's rankings
    totals = {}
    simSums = {}
    rankings_list =[]
    for other in dataset:
        # don't compare me to myself
        if other == person:
            continue
        sim = pearson_correlation(person,other)

        # ignore scores of zero or lower
        if sim <=0: 
            continue
        for item in dataset[other]:

            # only score movies i haven't seen yet
            if item not in dataset[person] or dataset[person][item] == 0:

            # Similrity * score
                totals.setdefault(item,0)
                totals[item] += dataset[other][item]* sim
                # sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+= sim

        # Create the normalized list

    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    rankings.sort()
    rankings.reverse()
    # returns the recommended items
    recommendataions_list = [recommend_item for score,recommend_item in rankings]
    return recommendataions_list[:maxnum]

In [12]:
Reviews = {}
def printReview(x):
    bid = x['business_id']
    uid = x['user_id']
    text = str(x['stars'])+" "+x['text']
    
    if bid not in Reviews.keys():
        Reviews[bid]= {} 
    Reviews[bid][uid] = text
    
review_df.apply(lambda x: printReview(x), axis=1)

0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
5996966    None
5996967    None
5996968    None
5996969    None
5996970    None
5996971    None
5996972    None
5996973    None
5996974    None
5996975    None
5996976    None
5996977    None
5996978    None
5996979    None
5996980    None
5996981    None
5996982    None
5996983    None
5996984    None
5996985    None
5996986    None
5996987    None
5996988    None
5996989    None
5996990    None
5996991    None
5996992    None
5996993    None
5996994    None
5996995    None
Length: 5996996, dtype: 

In [48]:
bids = user_recommendations("msQe1u7Z_XuqjGoqhB0J5g",3)
bids

['bG-JFEQjr0pAcgdz7eHkZw', 'Z8XyPYoxBSNpTNAehDbWLQ', 'VeiL_tgw7dsl-7IcnOsh0g']

In [14]:
Reviews["zomc4vlPucjG4lD1fglOkw"]

{'SB6WZI5iCB4zjXjDED01kA': '5 Nothing like a relaxing baseball game after a long day of work.  Plenty of tickets every night that I dont have to rush to the game.  Tons of promos going on to make the night even better!',
 'DW2D-2b706BqPcvJxKx4rA': '5 This is a fun night. If you like baseball...or just looking for something to do to keep away from the tables for a couple of hours...this is the place. The 51s are a AAA team affiliated with the NY Mets. Good seats are $15, many less expensive seats are available too. Seating is available on the lawn just outside of the right and left filed foul poles. Very kid friendly too. Give it a shot. Enjoy!',
 'PMzMA8lz3TSQGnSSxbg3yA': "4 I wasn't sure what to expect, having grown up going to baseball games at Qualcomm (previous home to the San Diego Padres), Dodger Stadium and AT&T Park.\n\nThe field at Cashman is actually really nice.  It's obviously smaller than larger stadiums, and the crowd is much smaller.  But there's lots of pros here, inclu

In [36]:
business_df[business_df['business_id']=='VeiL_tgw7dsl-7IcnOsh0g']

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
4587,"9460 S Eastern Ave, Ste 130","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",VeiL_tgw7dsl-7IcnOsh0g,"Sushi Bars, Restaurants, Japanese",Las Vegas,"{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:...",1,36.017617,-115.118231,Sushi Hiro,Southeast,89123,257,4.5,NV


In [18]:
with open('/Users/jieyizhang/Downloads/yelp_dataset/yelp_academic_dataset_user.json') as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

user_df = pd.DataFrame(data)

user_df[user_df['user_id']=='msQe1u7Z_XuqjGoqhB0J5g']

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
12988,3.31,2,0,2,1,0,0,0,0,0,...,1,None,1,"m_93Qy3Yhz31khEQVzqmqA, fNye_2mxMdQt7c4TFQdzww...",1,Melissa,11,4,msQe1u7Z_XuqjGoqhB0J5g,2011-02-24


In [21]:
review_df[review_df['user_id']=='msQe1u7Z_XuqjGoqhB0J5g']

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
2,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
3,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
4,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,0,msQe1u7Z_XuqjGoqhB0J5g
5,vgfcTvK81oD4r50NMjU2Ag,0,2011-02-25,0,pfavA0hr3nyqO61oupj-lA,1,This place sucks. The customer service is horr...,2,msQe1u7Z_XuqjGoqhB0J5g
6,AxeQEz3-s9_1TyIo-G7UQw,0,2011-10-10,0,brokEno2n7s4vrwmmUdr9w,5,"If you like Thai food, you have to try the ori...",1,msQe1u7Z_XuqjGoqhB0J5g
7,zdE82PiD6wquvjYLyhOJNA,0,2012-04-18,1,kUZWBVZvhWuC8TWUg5AYyA,5,AMAZING!!!\n\n I was referred here by a friend...,0,msQe1u7Z_XuqjGoqhB0J5g
8,EAwh1OmG6t6p3nRaZOW_AA,0,2011-02-25,0,wcqt0III88LEcm19IxFFyA,4,Ribs = amazing\n2 hour wait time= not so amazi...,0,msQe1u7Z_XuqjGoqhB0J5g
9,atVh8viqTj-sqDJ35tAYVg,1,2012-11-09,2,LWUtqzNthMM3vpWZIFBlPw,2,"Food is pretty good, not gonna lie. BUT you ha...",1,msQe1u7Z_XuqjGoqhB0J5g


In [38]:
business= business_df[['business_id','categories','city','review_count','stars']]
review_business = reviews.merge(business, left_on='business_id', right_on='business_id', how='left')

user = user_df[['user_id','review_count']].drop_duplicates()
review_business = review_business.merge(user, left_on='user_id', right_on='user_id', how='left')

review_business.head()

,user_id,business_id,stars_x,categories,city,review_count_x,stars_y,review_count_y
0,msQe1u7Z_XuqjGoqhB0J5g,iCQpiavjjPzJ5_3gPD5Ebg,2,"Pizza, Restaurants",Las Vegas,4078,4.0,11
1,msQe1u7Z_XuqjGoqhB0J5g,pomGBqfbxcqPv14c3XH-ZQ,5,"Restaurants, Mexican, Bars, Nightlife",Las Vegas,1125,4.0,11
2,msQe1u7Z_XuqjGoqhB0J5g,jtQARsP6P-LbkyjbO1qNGg,1,"American (New), Barbeque, Restaurants",Las Vegas,115,3.5,11
3,msQe1u7Z_XuqjGoqhB0J5g,elqbBhBfElMNSrjFqW3now,2,"Restaurants, Thai",Las Vegas,601,4.0,11
4,msQe1u7Z_XuqjGoqhB0J5g,Ums3gaP2qM3W1XcA5r6SsQ,5,"Restaurants, Desserts, Asian Fusion, Mexican, ...",Las Vegas,241,4.0,11


In [58]:
LV_usr = review_business[(review_business['city']=='Las Vegas')&(review_business['review_count_y']>=30)&(review_business['review_count_y']<3000)]
LV_usr = LV_usr.user_id.to_frame()
LV_usr=LV_usr.drop_duplicates()
LV_usr = LV_usr.reset_index(drop=True)
LV_usr

,user_id
0,s5LYD5-cU8ji2F-7yCYfew
1,FHwl6kaKdD5-KAnfVb_pKQ
2,AuIK5tF2GjO7SftHawTLKw
3,oZOBAtPYVUq724Wywoh5kg
4,D1_nrBr4dOrs7M82OaBRwQ
5,qr9hmaIxq1z9ycgT28PpRw
6,dEgO4_n8thLMbG1dKfFMHg
7,bsH9WdGVrFOjMlpDBNkWIA
8,1cd1TjQEynOjhBT-3Qnsjg
9,0wvlpzuR4mnZfMcoG8N31Q


In [78]:
dictc = {}
LV_samples = LV_usr.sample(500)
for i in range(len(LV_samples)):
    this = LV_samples['user_id'].iloc[i]
    recommend_list = user_recommendations(this,5)
    for r in recommend_list:
        if(r in dictc):
            dictc[r] += 1
        else:
            dictc[r] = 1
            
dictc

{'zK3s8x6B6KXSe4BOa407Vw': 10,
 'vu4DAmaeEK_cX1b6bDxn9Q': 1,
 'uuoNGJDHqbeuxV_zpq9vZQ': 2,
 'tYJg19wSFCTsEjlKhinVDw': 1,
 'sN3jqoj6oG10zqwjjGY0Dw': 1,
 'zsQk990PubOHjr1YcLkQFw': 3,
 'zoODlH40edpJYLPLkHilNA': 31,
 'zk3WJ9zM07RU0V0LvN93RQ': 3,
 'zdbTAij1bUDniEUvT_WAyA': 7,
 'zdE82PiD6wquvjYLyhOJNA': 17,
 'xn2wBz0SmCo5J6y10NK5ew': 3,
 'wHwHmozyqtIE5U3uTtvpuA': 1,
 'pxYB4ygKR-wkK5mwoJd8AQ': 2,
 'kzh7iDNd8Qwio2w3biAHGw': 2,
 'etid3CozFvEOFcvDwHXsDw': 2,
 'zcScEL0WEdFkROcnz5379g': 11,
 'zHHNbjg7Glc_fomaBdXPbQ': 1,
 'zGAFnGHxoldvBFH7KVKwaw': 3,
 'z8XIldYr65CKOz8yOI5R8A': 3,
 'z5FlFum4dsnhEgAJrBuO9Q': 2,
 'zwNC-Ow4eIMan2__bS9-rg': 4,
 'zqcGg4GKVK89NXp0rzJmPg': 6,
 'zdqy_whZwbyyMe-WdedYEA': 2,
 'zIjXNOEPub7T-NwQWPREMA': 1,
 'zjBJPx2Hm-cx1xfeIjcKHQ': 2,
 'z0J5ulstB5vqVPxMoaA2YA': 1,
 'yefpHM1CfpQ0C9zFBYAdXA': 2,
 'yX2Iu4--Z6QK7kfEvvpgHg': 1,
 'xXfBlr0GpwHs5cGG5SGTGw': 1,
 'y_1DkpqdtWreG6RB88yooA': 19,
 'xwIJGDiwUz5aorokYKfxIQ': 1,
 'xko9a0myoKffQMzz96O2jA': 1,
 'xIsjwmr-3Hr4bcznR1kjpw': 1,
 'whs

In [ ]:
LV_rocommendation = pd.DataFrame.from_dict(dictc, orient='index').reset_index()
LV_rocommendation.columns=['business_id','count']

In [ ]:
LV_rocommendation.sort_values(by=['count'],ascending=False)
LV_top10 = LV_rocommendation
LV_top10 = LV_top10.merge(business,left_on='business_id', right_on='business_id', how='left')

In [107]:
LV_top = LV_top10.sort_values(by=['count'],ascending=False)

In [102]:
Trt_usr = review_business[(review_business['city']=='Toronto')&(review_business['review_count_y']>=30)&(review_business['review_count_y']<3000)]
Trt_usr = Trt_usr.user_id.to_frame()
Trt_usr=Trt_usr.drop_duplicates()
Trt_usr = Trt_usr.reset_index(drop=True)
Trt_usr

,user_id
0,YHWsLBS8jzZiPjKHMFOaAA
1,YgavGxfAdjhkkbwlAY_9ZQ
2,UGW-9bbBEB3eP1o6mWD_WA
3,kBNFdviedCPFWyR-wVaAzw
4,jczg3Je7CYjh6HOsW5_veA
5,aBXJK7fk2GDhDmgJEKiBeA
6,95KCaXFJEGla4AK7ETBmWw
7,Fpy6MlIcTVTS3jTluhNxgA
8,PkRFSQgSfca9Tamq7b2LdQ
9,Gcr2xLgfVlQBx6kabt2k_A


In [ ]:
dictc = {}
Trt_samples = Trt_usr.sample(500)
for i in range(len(Trt_samples)):
    this = Trt_samples['user_id'].iloc[i]
    recommend_list = user_recommendations(this,5)
    for r in recommend_list:
        if(r in dictc):
            dictc[r] += 1
        else:
            dictc[r] = 1
            
dictc

Trt_rocommendation = pd.DataFrame.from_dict(dictc, orient='index').reset_index()
Trt_rocommendation.columns=['business_id','count']
Trt_rocommendation.sort_values(by=['count'],ascending=False)
Trt_top10 = Trt_rocommendation
Trt_top10 = Trt_top10.merge(business,left_on='business_id', right_on='business_id', how='left')

In [106]:
Trt_top = Trt_top10.sort_values(by=['count'],ascending=False)

In [112]:
rest=0
muse=0
cin=0
doc=0
shop=0
beauty=0

for i in range(len(Trt_top)):
    cats = Trt_top['categories'].iloc[i]
    print(cats)
    try:
        if 'Restaurants' in cats:
            rest = rest + 1
        if 'Museums' in cats:
            muse = muse + 1
        if 'Cinema' in cats:
            cin = cin+1
        if 'Doctors' in cats:
            doc=doc+1
        if 'Shopping' in cats:
            shop=shop+1
        if 'Beauty' in cats:
            beauty = beauty + 1
    except:
        pass
        
print('Restaurants ',str(rest),' Museums',str(muse),' cin',str(cin),' doc',str(doc),' shop',str(shop),' beauty',str(beauty))

Burgers, Restaurants
Fast Food, Restaurants, Burgers
Ice Cream & Frozen Yogurt, Food
Public Services & Government, Libraries
Active Life, Axe Throwing, Sports Clubs
Restaurants, Coffee & Tea, Food, Cafes
Parks, Active Life
Coffee & Tea, Food
Local Flavor, Yelp Events
Specialty Food, Ethnic Food, Food, Restaurants, Caribbean
Financial Services, Payroll Services, Professional Services, Financial Advising, Tax Services, Accountants
Local Flavor, Yelp Events
Event Planning & Services, Hotels & Travel, Hotels
Laser Eye Surgery/Lasik, Health & Medical, Optometrists
Desserts, Middle Eastern, Restaurants, Food, Lebanese
Day Spas, Chiropractors, Health & Medical, Massage, Beauty & Spas, Medical Spas
Restaurants, Sushi Bars, Japanese
Chinese, Restaurants, Specialty Food, Dim Sum, Food
Street Vendors, Food, Sandwiches, Restaurants, Breakfast & Brunch
Bakeries, Food
Restaurants, Mediterranean
Creperies, Ice Cream & Frozen Yogurt, Restaurants, Food, Italian
Food, Bagels, Coffee & Tea
Orthodontists,

Food, Bakeries, Turkish, Restaurants, Patisserie/Cake Shop
Specialty Food, Food, Seafood Markets
Mediterranean, Sandwiches, Vegetarian, Middle Eastern, Restaurants
Restaurants, Sushi Bars
Home Services, Handyman, Lighting Fixtures & Equipment, Local Services, Electricians
Automotive, Parking
Gyms, Sports Clubs, Active Life, Fitness & Instruction, Trainers
Food, Bakeries
Parks, Active Life, Hiking
Donuts, Food, Patisserie/Cake Shop, Ice Cream & Frozen Yogurt, Bakeries
Event Planning & Services, Caterers, Restaurants, Pizza
Cocktail Bars, Bars, Nightlife, Tiki Bars
Beauty & Spas, Hair Salons
Himalayan/Nepalese, Restaurants
Nightlife, Bars, Restaurants, American (Traditional)
Pizza, Italian, Restaurants
Active Life, Parks
Restaurants, Canadian (New)
Massage, Massage Therapy, Nail Salons, Health & Medical, Eyelash Service, Hair Removal, Beauty & Spas, Skin Care
Cocktail Bars, Steakhouses, Food, Beer, Wine & Spirits, Nightlife, Seafood, Beer Bar, Asian Fusion, Wine Bars, American (Tradition

In [114]:
np.mean(business['stars'])

3.6315504817251965

In [116]:
np.mean(Trt_top['stars'])

4.140984908657665

In [117]:
np.mean(LV_top['stars'])

4.178650442477876